In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import cv2

from video_utils import *
from frame_utils import *
from foreground_segmentation import *

from tqdm.notebook import tqdm

from skimage.color import rgb2gray
from skimage.filters import sobel
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float, img_as_ubyte

%load_ext autoreload
%autoreload 2

### Image Pre-Processing (Optional)

In [ ]:
H = 1080 // 3
W = 1920 // 3

vid_frames = load_video('videos/1_Basketball.mp4', H, W)
print(vid_frames.shape)

In [ ]:
n_clusters = 500
compactness = 20

In [ ]:
fig, ax = segment_frames(vid_frames, [0,100,200,300], 2, 2, n_clusters, compactness)
plt.show()

In [ ]:
vid_labels = preprocess_video(vid_frames[:100], tqdm, n_clusters, compactness)

In [ ]:
np.save('preprocess_cache/1_Basketball.npy', vid_labels)

In [ ]:
# vidwrite_from_numpy('outputs/1_Basketball.mp4', vid_frames)

### Visualizing Segmented Frames

In [ ]:
H = 1080 // 3
W = 1920 // 3

vid_frames = load_video('videos/1_Basketball.mp4', H, W)[:100]
label_frames = np.load('preprocess_cache/1_Basketball.npy')

In [ ]:
F, H, W, C = vid_frames.shape

segmented_vid_frames = np.zeros_like(vid_frames)

for f in tqdm(range(F)):
    segmented_frame = get_segmented_image(vid_frames[f], label_frames[f])
    segmented_vid_frames[f] = img_as_ubyte(segmented_frame)
    
vidwrite_from_numpy('outputs/1_Baketball_Segmented.mp4', segmented_vid_frames)

In [ ]:
plt.figure()
fig, ax = plt.subplots(2, 2, figsize=(7,4))

I = 80

ax[0,0].imshow(segmented_vid_frames[I])
ax[0,1].imshow(segmented_vid_frames[I+1])
ax[1,0].imshow(segmented_vid_frames[I+2])
ax[1,1].imshow(segmented_vid_frames[I+3])

for a in ax.ravel():
    a.set_axis_off()

In [ ]:
jaccand_threshold = 0.4

mpixels = get_motion_superpixels(
    label_frames[80], label_frames[81], jaccand_threshold
)

In [ ]:
motion_superpix_img = np.copy(segmented_vid_frames[81])

for label in mpixels:
    label_pix_x, label_pix_y = np.where(label_frames[81] == label) 
        
    motion_superpix_img[label_pix_x,label_pix_y,:] = np.array([0,255,0])
    
plt.figure()
fig, ax = plt.subplots(1, 3, figsize=(7,4))

ax[0].imshow(segmented_vid_frames[80])
ax[1].imshow(segmented_vid_frames[81])
ax[2].imshow(motion_superpix_img)

plt.show()